<a href="https://colab.research.google.com/github/Saif-Shines/pk-cookbook/blob/tracing-assistants-api-klyst-176/product/tracing-assistants-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracing Assistants API

OpenAI’s Assistants API lets you access tools and persist threads to perform various tasks.

Accomplishing a task means multiple moving parts:



* Create an `Assistant` that encapsulates the base model, instructions, tools, hyperparameters, and necessary (context) documents.
* Create a `Thread` that represents an stateful conversation.
* Execute `Runs` on top of `Assistant`s and `Thread`s.

This notebook provides a step by step guide to trace all the requests (using [Portkey](https://www.portkey.ai/)) to accomplish a task—making it easy for troubleshooting and debugging.  


## 0. Install OpenAI and Portkey SDKs

In [ ]:
!pip install openai portkey-ai


Start by importing them

In [2]:
from openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
from google.colab import userdata
import json, time

`PORTKEY_GATEWAY_URL` and `createHeaders` are necessary imports that lets you instantiate `OpenAI` and enable request tracing. Whereas, `json`, `time`, `userdata` are utilities for better logging and handling environment variables (in google colab).

In [8]:


def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

## 1. Enable tracing at instantiation

Tracing is made possible by enabling OpenAI SDK send requests through Portkey.



In [3]:
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY'), # instead virtual key is respected
    base_url=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        provider="openai",
        api_key=userdata.get('PORTKEY_API_KEY'),
        virtual_key=userdata.get('OPENAI_VIRTUAL_KEY'),
        trace_id="assistants"
    )
)

Portkey’s gateway URL is set as `base_url` where requests are sent through.

The `default_headers` contains the headers that directs requests to the target Large Langauage Models (LLMs).

* **Trace ID:** Specify any desirable string to trace through the logs.
* **Virtual Keys:** Alternative to using original API keys once secured in the Portkey app. See [docs](https://portkey.ai/docs/product/ai-gateway-streamline-llm-integrations/virtual-keys).
* **API Key:** Specify the [Portkey API key](https://portkey.ai/docs/welcome/make-your-first-request#id-1.-get-your-portkey-api-key).

Among the top level `api_key` and `virtual_key` (in default headers), the `virtual_key` (when present) is respected.


##  2. Tools: Prepare for File Search

This notebook demonstrates how to trace as you use Assistants API to retrieve an PDF document and generate a tweet thread based on it.

Upload the PDF document and get an File ID to reference it in the Vector Store.

In [4]:
file = client.files.create(
    file=open("prompt-partials.pdf", "rb"), # upload in local any PDF
    purpose="assistants"
  )

file_id = file.id

print(file_id)

file-cdoQlBYIVayRBJoqqLUAk6qA


In [5]:
vector_store = client.beta.vector_stores.create(
  name="Documents",
  file_ids=[file_id]
)

print(vector_store)

VectorStore(id='vs_El8BQ75OkBpcbdYSfCpgFEJB', created_at=1714407352, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=1, total=1), last_active_at=1714407352, metadata={}, name='Documents', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)


## 3. Create an `Assistant` and a `Thread`

Instruct the assistant to convert the document into twitter threads.


In [6]:
assistant = client.beta.assistants.create(
    name="Y Threads Converter",
    instructions="You are a helpful assistant. Given a set of files, you extract the most interesting information and restructure it into Threads format for Twitter without asking any further questions.",
    model="gpt-4-1106-preview",
    tools=[{"type": "file_search"}], # See supported tools on OpenAI docs
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

assistant_id = assistant.id
print(assistant_id)

asst_NXAwYqPblhNMWee7D1yLler5


The `show_json(..)` utility helps to log response in JSON for better readability.

In [7]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))


Next up create a thread and add a user message.

In [8]:
thread = client.beta.threads.create()

thread_id = thread.id

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a X Thread",
)
show_json(message)

{'id': 'msg_qYldKBB2WYUncBAxH3gf9QU5',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': 'Create a X Thread'},
   'type': 'text'}],
 'created_at': 1714407520,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_vhCz2rOYT8qWwws1H4XaczfP'}

## 4. Perform the Task with a `Run`

Creating a `run` means to allow an `assistant` to execute on a `thread`. This asynchronous operation returns `run`’s metadata immediately where `status` is set to `queued`.

The `wait_on_run(..)` utility ensures the `run` is `completed` by polling the `run` in a loop.

In [10]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

Execute a `run`

In [11]:
run = client.beta.threads.runs.create(
  thread_id=thread_id,
  assistant_id=assistant_id
)

run_id = run.id

run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_bWUOwm5oD3dkHIb3KLMa538b',
 'assistant_id': 'asst_NXAwYqPblhNMWee7D1yLler5',
 'cancelled_at': None,
 'completed_at': 1714407639,
 'created_at': 1714407603,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a helpful assistant. Given a set of files, you extract the most interesting information and restructure it into Threads format for Twitter without asking any further questions.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1714407603,
 'status': 'completed',
 'thread_id': 'thread_vhCz2rOYT8qWwws1H4XaczfP',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search'}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 529,
  'prompt_tokens': 2272,
  'total_tokens': 2801},
 'temperature': 1.0,
 'top_p': 1.0,


## 5. List the messages in the thread

Depending on the instructions and user messages, the assistant may perform the task or ask answers for more questions. Prompt the user for additional inputs to give assistant the sufficient context.

In this example, the instructions include not to ask further questions. List the messages on the thread that includes the tweets.


In [12]:
thread_messages = client.beta.threads.messages.list(thread_id)
print(thread_messages.data[0].content[0].text.value)
# show_json(thread_messages)

Creating a Twitter Thread using Portkey Prompt Partials:

🧵1/ X Thread:
Let's dive into how prompt partials revolutionize prompt creation on Portkey app. Imagine crafting diverse prompts with a common core—simple, efficient, and effective!

2/ Prerequisites:
Before harnessing the power of prompt partials, ensure you've integrated OpenAI with your Portkey account. Secure your API keys and prep for a smooth workflow: https://www.portkey.ai.

3/ Installing Portkey SDK:
Jumpstart your journey with the following command:
```
npm install portkey-ai
```

4/ Crafting Partial Prompts:
A building block for multipurpose prompts—define once, use anywhere!

Here’s a guide to creating a partial prompt:
Navigate to Prompts > Prompt Partials (Tab) > Create.

5/ Mustache Syntax for Variables:
Employ mustache syntax to dynamically insert values into your prompts. A unique partial ID locks the blueprint for reuse.

6/ Designing Prompt Templates:
The Prompt Playground on Portkey lets you experiment and fi

## 6. Tracing and Logging

Input `assistants` as **Trace Id **in the search bar to list all the logs specific to the current Assistant API. The `PATH` column indicates the nature of the operations - creating files, runs, messages, retrieving runs, and listing messages.

![](https://raw.githubusercontent.com/Portkey-AI/portkey-cookbook/be59727ffedf76be581c24689cd27cc596f5b521/product/images/tracing-assistants-api/1-tracing-assistants-api.gif)

All the request and response logs are available for you to quickly identify and investigate any issues when you encounter them.
